In [115]:
import itertools
import re
import pandas as pd

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

In [122]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--headless')
driver=webdriver.Chrome(r'C:\Users\User\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe', options=options)
driver.get(r'https://www.nhl.com/news/nhl-fantasy-hockey-top-250-rankings-drafts-players-2020-21/c-281505474?tid=277729150')
data=driver.find_element_by_xpath("/html/body/div[3]/div[3]/div[2]/div/article[1]/div[4]/div[2]/p[12]").text

In [123]:
data=data.replace('\n',',').split(',')

In [124]:
names=data[::3]
ranking=[i.split('.')[0].strip() for i in names]
player=[i.split('.')[1].strip() for i in names]

position=data[1::3]
position=[i.strip() for i in position]

team=data[2::3]
team=[i.split()[0].strip('*') for i in team]

In [125]:
nhl_rankings = pd.DataFrame(list(zip(ranking, player, position, team)),
                  columns = ['Ranking' , 'Player', 'Position', 'Team'], index=range(len(ranking)))

In [127]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--headless')
driver=webdriver.Chrome(r'C:\Users\User\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe', options=options)
driver.implicitly_wait(10)
driver.get(r'https://hockey.fantasysports.yahoo.com/hockey/18910/draftresults')
driver.find_element_by_xpath("/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/form/div[1]/div[3]/input").send_keys('afriedland94@gmail.com', Keys.ENTER)
driver.find_element_by_id("login-passwd").send_keys('pw', Keys.ENTER)
data=driver.find_element_by_xpath("/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section[1]/div/div/div[3]/section/div").text

In [128]:
data=data.split('\n')
data=data[:273]

In [129]:
round=data[::39]
round=list(itertools.chain.from_iterable(itertools.repeat(i, 12) for i in round))

data=[i for i in data if i not in data[::39]]
data=[i for i in data if i not in data[9::38]]
data=[i for i in data if i not in data[36::37]]

player=data[::3]
player=[re.sub(r'^[0-9]+.\s', '', i).strip(' \ue03e') for i in player]

items=data[1::3]
team=[i[1:4] for i in items]
position=[i.split('- ')[1].strip(')') for i in items]

fantasy_team=data[2::3]

In [130]:
picks = pd.DataFrame(list(zip(round, player, position, team, fantasy_team)),
                  columns = ['Round' , 'Player', 'Position', 'Team', 'Fantasy_Team'], index=range(len(round)))

In [133]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--headless')
driver=webdriver.Chrome(r'C:\Users\User\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe', options=options)
driver.get(r'https://www.fantasypros.com/nhl/rankings/overall.php')
data=driver.find_element_by_xpath("/html/body/div[2]/div[5]/div/div[1]/div/div[3]/table").text

In [134]:
data=data.split('\n')[9:]
data=[re.sub('^[0-9]+ ','',i).split(')')[0].strip() for i in data]
player=[i.split('(')[0].strip() for i in data]
team=[i.split('(')[1].strip()[:3].strip() for i in data]
position=[i.split('(')[1].strip().split('- ')[1] for i in data]
ranking=[i+1 for i in range(len(data))]

In [135]:
fp_rankings = pd.DataFrame(list(zip(ranking, player, position, team)),
                  columns = ['Ranking' , 'Player', 'Position', 'Team'], index=range(len(ranking)))

In [136]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--headless')
driver=webdriver.Chrome(r'C:\Users\User\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe', options=options)
driver.get(r'https://dobberhockey.com/2020/11/03/top-300-keeper-league-skaters-november-2020/')
data=driver.find_element_by_xpath("/html/body/div[2]/div[2]/main/div/section/div/div/div[3]/div/div[1]/div/div[1]/figure[2]/table").text

In [137]:
data=data.split('\n')[1:]
data=[re.sub('^[0-9]+ ','',i) for i in data]
player=[re.split('([A-Z][A-Z]+)',i)[0].strip() for i in data]
team=[re.split('([A-Z][A-Z]+)',i)[1].strip() for i in data]
ranking=[i+1 for i in range(len(data))]

In [138]:
dobber_rankings = pd.DataFrame(list(zip(ranking, player, team)),
                  columns = ['Ranking' , 'Player', 'Team'], index=range(len(ranking)))

In [139]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--headless')
driver=webdriver.Chrome(r'C:\Users\User\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe', options=options)
driver.get(r'https://www.espn.com/fantasy/hockey/story/_/id/30389020/top-300-player-rankings-2020-21-fantasy-hockey-season')
data=driver.find_element_by_xpath("/html/body/div[5]/section/section/div/section[2]/article[1]/div/div[2]/p[19]").text

In [140]:
data=re.sub('\n', '', data)
data=re.split('[0-9]\.', data)[1:]
player=[i.split(',')[0].strip() for i in data]
team=[i.split(',')[1].split(' (')[0].strip() for i in data]
ranking=[i+1 for i in range(len(data))]

In [141]:
espn_rankings = pd.DataFrame(list(zip(ranking, player, team)),
                  columns = ['Ranking' , 'Player', 'Team'], index=range(len(ranking)))

In [164]:
final=pd.merge(nhl_rankings, fp_rankings, how='right', left_on='Player', right_on='Player', suffixes=('_nhl', '_fp'))
final=pd.merge(dobber_rankings, final, how='right', left_on='Player', right_on='Player', suffixes=('_dobber', ''))
final=pd.merge(espn_rankings, final, how='right', left_on='Player', right_on='Player', suffixes=('_espn', ''))

In [171]:
final=final[['Player', 'Team_fp', 'Position_fp', 'Ranking_fp', 'Ranking_nhl', 'Ranking_espn', 'Ranking']]
final.rename(columns={'Team_fp': 'Team', 'Position_fp': 'Position', 'Ranking': 'Ranking_dobber'}, inplace=True)

In [183]:
final['Average_Ranking']=final[['Ranking_fp', 'Ranking_nhl', 'Ranking_espn', 'Ranking_dobber']].mean(axis=1).round(2)
final.drop(['Ranking_fp', 'Ranking_nhl', 'Ranking_espn', 'Ranking_dobber'], axis=1, inplace=True)

In [192]:
final.sort_values(by=['Average_Ranking'], inplace=True)

In [194]:
chosen=final.Player.isin(picks.Player)
final['Chosen']=chosen
available=final[final.Chosen==False]

In [197]:
final.to_csv(r'C:filepath.csv')